## LOAD SOLUTION FILES

In [1]:
import os
import pickle
import pandas as pd
from tf_attention import *
from tf_helpers import *

### Test Encoder

In [6]:
# print(torch.cuda.is_available())
%run tf_helpers.py
import tensorflow as tf

size = 32
sequence_length = 625
input_dim = 9
output_dim = 3

path_encoded = path_encoder()
# X = []
# Y = []
network = []
for i in tqdm(range(size)) :
    file_name = f"Output/5by5_Data{i}"
    # x, y = generate_xy(file_name, path_encoded, ' ')
    # X.append(x)
    # Y.append(y)

    file = open(file_name, "rb")
    stat = pickle.load(file)
    file.close()
    net = stat["data"]["network"]
    Graph = get_graphTensor(net)
    network.append(Graph)
# X = tf.stack(X, axis=0)
# Y = tf.stack(Y, axis=0)
    

  0%|          | 0/32 [00:00<?, ?it/s]

In [7]:
a = pd.DataFrame(network[0].numpy())
a.head()

,0,1,2,3,4
0,1.0,2.0,2000.0,40.0,0.6
1,2.0,1.0,2000.0,40.0,0.6
2,1.0,6.0,2000.0,40.0,0.6
3,6.0,1.0,2000.0,40.0,0.6
4,2.0,3.0,2000.0,40.0,0.6


In [5]:
in_mask = tf.cast(tf.not_equal(X, 0), dtype=tf.float32) # TensorShape([32, 625, 9])
X = X * in_mask # TensorShape([32, 625, 9])
out_mask = tf.cast(tf.not_equal(Y, 0), dtype=tf.float32)
Y = Y * out_mask
out_mask.shape

TensorShape([32, 625, 3])

In [3]:
%run tf_attention.py

encoder = Encoder(input_dim=9, d_model=512, N=6, heads=8, dropout=0.1)
encoder_output, attention_scores = encoder(X)
print(encoder_output.shape)

(32, 625, 3)


In [35]:
pd.DataFrame(encoder_output[0].numpy()).head()

,0,1,2
0,0.000000,2.025494,0.000000
1,0.000000,2.343900,0.701559
2,0.768207,0.641126,0.000000
3,0.000000,2.381319,0.000000
4,0.000000,1.976538,0.000000


### Test Decoder

In [13]:
%run tf_attention.py
decoder = Decoder(output_dim=3, d_model=512, N=6, heads=8, dropout=0.1)
decoder_out, scores = decoder(Y, encoder_output)

In [14]:
pd.DataFrame(decoder_out[0].numpy()).head()

,0,1,2
0,-0.430916,1.381320,-0.950404
1,1.412972,-0.672857,-0.740115
2,1.409592,-0.795832,-0.613760
3,1.413325,-0.726242,-0.687083
4,-0.370846,1.366674,-0.995829


### Test Transformer

In [20]:
%run tf_attention.py

transformer = Transformer(9,3,512,6,8,0.1)
transformer.eval()
out = transformer(X, Y)
pd.DataFrame(out[0].numpy()).head()

,0,1,2
0,0.829380,0.654449,-0.008664
1,0.073648,0.939672,1.811048
2,0.082913,0.951749,1.820750
3,0.090501,0.961588,1.828586
4,0.826528,0.647151,-0.019008


### Extract OD demand and path set (X and Y)
X: OD demand, graph (link feature), path, link-path adj \
Y: path flow

In [4]:
%run tf_attention.py
%run tf_helpers.py 

class Dataset:
    def __init__(self, size, standard_norm, start_from=0):
        self.path_encoded = path_encoder()  # Get path encode dictionary
        self.X = []
        self.Y = []

        for i in tqdm(range(size)):
            file_name = f"Output/5by5_Data{start_from+i}"
            x, y = generate_xy(file_name, self.path_encoded, standard_norm)
            self.X.append(x)
            self.Y.append(y)
        
        self.X = tf.stack(self.X, axis=0)
        self.Y = tf.stack(self.Y, axis=0)
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

    def to_tf_dataset(self, batch_size):
        dataset = tf.data.Dataset.from_tensor_slices((self.X, self.Y))
        dataset = dataset.shuffle(buffer_size=len(self.X)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return dataset 
    

In [5]:
# %run helpers.py
batch_size = 32
train_size = 1000
val_size = 500
standard_norm = 'normalize'

train_dataset = Dataset(train_size, standard_norm)
train_data_loader = train_dataset.to_tf_dataset(batch_size)

val_dataset = Dataset(val_size, standard_norm, start_from=train_size)
val_data_loader = val_dataset.to_tf_dataset(batch_size)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

### TRY TRAINING AND VALIDATING

In [5]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
%run tf_attention.py

# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# device = tf.device('gpu' if tf.test.is_gpu_available() else 'cpu')
device = 'cpu'

model = Transformer(input_dim=input_dim, output_dim=output_dim, d_model=512, N=2, heads=8, dropout=0.1)

epochs = 100
gradient_accumulation_steps = 4
learning_rate = 0.001

loss_fn = MeanSquaredError()
optimizer = Adam(learning_rate=learning_rate)

train_losses = []
val_losses = []

# Training loop
with tqdm(total=epochs, unit="epoch") as pbar:
    for epoch in range(epochs):    
        # Training phase
        total_train_loss = 0
        for src, trg in train_data_loader:
            with tf.device(device):
                with tf.GradientTape() as tape:
                    output = model(src, trg)
                    loss = loss_fn(trg, output)
                
                # Backpropagate and update the model
                gradients = tape.gradient(loss, model.trainable_variables)
                optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                total_train_loss += loss.numpy()
        
        # Validation phase
        model.eval()
        total_val_loss = 0
        for src, trg in val_data_loader:
            # Move the batch to the device
            with tf.device(device):
                # Forward pass
                output = model(src, trg)
                loss = loss_fn(trg, output)
                total_val_loss += loss.numpy()
            
        pbar.update(1)
        pbar.set_description(f"Train Loss: {total_train_loss / len(train_data_loader):.4f}, Val Loss: {total_val_loss / len(val_data_loader):.4f}")
        
        train_losses.append(total_train_loss / len(train_data_loader))
        val_losses.append(total_val_loss / len(val_data_loader))
        
        # Print the training and validation losses
        print(f"Epoch: {epoch+1} - Train Loss: {total_train_loss/len(train_data_loader):.4f}, Val Loss: {total_val_loss/len(val_data_loader):.4f}")


  0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch: 1 - Train Loss: 122937.9526, Val Loss: 54378.4285
Epoch: 2 - Train Loss: 120377.7637, Val Loss: 54536.9238
Epoch: 3 - Train Loss: 121322.8350, Val Loss: 54098.4968
Epoch: 4 - Train Loss: 120784.8918, Val Loss: 54268.4746
Epoch: 5 - Train Loss: 120104.6985, Val Loss: 54246.6465
Epoch: 6 - Train Loss: 119604.8193, Val Loss: 53944.2224
Epoch: 7 - Train Loss: 120898.2888, Val Loss: 54225.2168
Epoch: 8 - Train Loss: 119720.0632, Val Loss: 54298.2280
Epoch: 9 - Train Loss: 121202.2576, Val Loss: 53919.2336
Epoch: 10 - Train Loss: 121297.1584, Val Loss: 54445.5315
Epoch: 11 - Train Loss: 120660.6868, Val Loss: 53933.6802
Epoch: 12 - Train Loss: 120054.6038, Val Loss: 54147.7688
Epoch: 13 - Train Loss: 121297.9155, Val Loss: 54275.1213
Epoch: 14 - Train Loss: 120526.8169, Val Loss: 54342.1501
Epoch: 15 - Train Loss: 119403.0864, Val Loss: 54370.5806
Epoch: 16 - Train Loss: 120615.0029, Val Loss: 54265.0337
Epoch: 17 - Train Loss: 120732.8069, Val Loss: 54111.0569
Epoch: 18 - Train Loss:

In [ ]:
def plot_loss(train_losses, val_losses, epochs):
    plt.figure(figsize=(12, 6))
    plt.plot(range(1, epochs+1), train_losses, label='Training Loss')
    plt.plot(range(1, epochs+1), val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

# Example usage
train_losses = [0.8, 0.6, 0.5, 0.4, 0.3]
val_losses = [0.9, 0.7, 0.6, 0.5, 0.4]
epochs = 5

plot_loss(train_losses, val_losses, epochs)